In [21]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from collections import defaultdict

import requests
from itertools import repeat
import pandas as pd
import os

In [22]:
options = webdriver.ChromeOptions()

prefs = {'profile.default_content_setting_values.geolocation' :2,
         'profile.managed_default_content_settings.images':2,
         'disk-cache-size': 4096}

options.add_experimental_option("prefs",prefs)

In [23]:
def combining_course_and_ies(cursos_list, players_list): 
    
    '''Receberá duas listas contendo informações com nome da instituição, curso ou localização, combinará os dois em uma lista
       e irá adicionar essa lista a outra lista'''
    
    
    combined_list = []
    # Loopando pela primeira lista
    for i in cursos_list:
        # Loopando pela segunda lista
        for j in players_list:
            # Combinando os dois valores em uma lista, separados por /
            combined_list.append([i + '/' + j])
   
    combined = [element.split('/') for sublist in combined_list for element in sublist]
    return combined

In [26]:
cursos = ['matematica', 'geografia', 'história']
for curso in cursos:
    inputing_values(curso)

In [25]:
links_to_crawl, failed_combinations, combinations_to_try_again = [], [], []


def inputing_values(curso):
    
    '''Inputa as informações de curso, ies e loc, da buscar e salva o link da página, caso haja cursos para aquela 
       IES naquela cidade, numa lista. Também cria uma lista com as combinações que não deram certo.'''
    
    # Instanciando o driver e passando o site
    drive = webdriver.Chrome(options=options)
    drive.get('https://querobolsa.com.br/')
    drive.maximize_window()
    
    
    try:
        # Clicando no botão para limpor a cidade
        drive.find_element_by_xpath('//div[@class="ui-input ui-autocomplete__input ui-input--with-clear-button"]/button').click()
        
        # Inputando o curso no campo curso
        input_curso = drive.find_element_by_xpath('//input[@placeholder="Digite o curso"]').send_keys(curso)
        time.sleep(2)
        # Selecionando o primeiro elemento da caixa 
        fill_curso = drive.find_element_by_xpath('//ul[@id="ui-autocomplete-632-list"]/li').click()
        
        time.sleep(1.5)
        
        # Clicando no botão de procurar
        search_button = drive.find_element_by_xpath('//button[@class="z-button search-unified__button js-filter-button z-button--brand z-button--medium"]/span').click()

        time.sleep(3.5)
        
        # Caso não encontre nenhum curso irá adicionar os inputs na lista failed_combinations e não pegará o url da página
        try:
            drive.find_element_by_xpath('//strong[@class="z-text z-text--medium z-text--major z-text--left"]')
            print('Não encontramos o curso de {} na instituição {} na cidade de {}.'.format(curso, ies, loc))
            
            drive.quit()
        # Caso encontre cursos, irá adicionar o url da página na lista links_to_crawl    
        except:    
            links_to_crawl.append(drive.current_url) 
            drive.quit()
            
    # Caso haja algum erro na excecução do código irá adicionar os inputs na lista combinations_to_try_again    
    except:
        drive.quit()
        
        
    return links_to_crawl, failed_combinations, combinations_to_try_again

In [28]:
for link in links_to_crawl:
    get_links_quero_bolsa(link)

In [27]:
link_list = []

def get_links_quero_bolsa(url):
    
    '''Irá receber os links da função inputing_values e irá passar por todas as páginas e pegará os links para as páginas
       do curso'''
    
    global link_list
    
    # Instânciando o webdriver
    wd = webdriver.Chrome(options=options)
    wd.get(url)
    wd.maximize_window()
    
    # Pegando todos os links da página
    links = wd.find_elements_by_xpath('//a[@href]')
    
    # Loopando através dos elementos da lista links
    for link in links:
        try:
            # Checando se há a substring '/cursos/' no link, caso tenha irá adicionar na lista link_list
            if '/cursos/' in link.get_attribute('href'):
                link_list.append(link.get_attribute('href'))
        except:
            pass
    try:
        # Selecionando o número da última página
        final_page = int(wd.find_elements_by_xpath('//ul[@class="z-pagination--desktop"]//li')[-1].text)

        n = 0
        # Loop para passar por todas as páginas 
        while n < final_page:

            try:
                # Clicando no botão de próxima página
                wd.find_elements_by_xpath('//a[@aria-label = "Próximo"]')[0].click() 

                time.sleep(1)
                # Pegando os links da página 
                links = wd.find_elements_by_xpath('//a[@href]')
                
                # Loopando através dos elementos da lista links
                for link in links:
                    # Checando se há a substring '/cursos/' no link, caso tenha irá adicionar na lista link_list
                    if '/cursos/' in link.get_attribute('href'):
                        link_list.append(link.get_attribute('href'))
                # Aumentando o valor de n
                n += 1

            except:
                wd.quit()
                return link_list
    except:
        wd.quit()
            
    wd.quit()
    
    return link_list

In [35]:
list(enumerate(set(link_list)))

[(0,
  'https://querobolsa.com.br/unp-universidade-potiguar/cursos/geografia/1452f8af-5773-466a-8128-dd86b7af225a'),
 (1, 'https://querobolsa.com.br/estacio/cursos/geografia'),
 (2,
  'https://querobolsa.com.br/unopar/cursos/matematica/025b6998-8295-49a7-93e0-d28abe3763cc'),
 (3,
  'https://querobolsa.com.br/anhembi-morumbi/cursos/geografia/bc718f90-09d4-47f7-84d5-67746851d734'),
 (4,
  'https://querobolsa.com.br/unp-universidade-potiguar/cursos/geografia/0a73e154-846e-4785-b2d5-eaff72185510'),
 (5,
  'https://querobolsa.com.br/unp-universidade-potiguar/cursos/geografia/96e7f9ce-07d4-46b0-b04e-bcbcab15a782'),
 (6,
  'https://querobolsa.com.br/unp-universidade-potiguar/cursos/matematica/b423a576-80fa-4b99-82dd-1fedc91b158e'),
 (7,
  'https://querobolsa.com.br/uniages-centro-universitario/cursos/matematica/ee3b4786-754e-4831-b79a-3b3cc1788a0e'),
 (8,
  'https://querobolsa.com.br/esbam-escola-superior-batista-do-amazonas/cursos/matematica/4732db1c-69e4-4acf-85c3-3c4ef526c50c'),
 (9,
  'ht

In [ ]:
for link in set(link_list):
    get_data(link)

In [46]:
data = pd.DataFrame(info_dic)
path = r'C:\Users\Victor Boscano\Desktop\Victor'
os.chdir(path)
data.to_excel('cursos_felipe.xlsx')

In [2]:
lis = [1, 2, 3]
lis + [5]


[1, 2, 3, 5]

In [29]:
# Pegando informação do curso principal

info_dic = defaultdict(list)
not_passed, avaliacao = [], []

def get_data(url):
    
    '''Receberá os urls da página do curso, retornados pela função get_links_quero_bolsa e pegará as informação relevantes'''    
    
    # Instânciando o webdriver
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    driver.maximize_window()
    
    # Caso haja problema na excecução do código abaixo irá adicionar o input a lista not_passed
    try:
        # Selecionando o título e transformando numa lista
        title = driver.find_element_by_xpath('//h1[@class="header__title"]').text.split('na ')
        # Selecionando a IES
        ies = title[-1]
        # Selecionando o Curso
        curso = title[0]
        
        # Pegando o valor dos cursos
        main_course_prices = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'offer-details-card-price__prices'))
        )
        # Preço com desconto
        main_preco_com_desconto = main_course_prices.text.split('R')[1].replace('R$', '')
        # Preço sem desconto
        main_preco_sem_desconto = main_course_prices.text.split('R')[2].replace('R$', '')
        
        # Selecionando o span onde estão as informações
        main_infos = driver.find_elements_by_xpath('//span[@class="offer-card__info-text"]/span[@class="offer-card__info-bold-text"]')
        # Separando as informações acima a sua categoria
        main_inicio, main_turno, main_duracao, main_nivel = main_infos[0].text, main_infos[1].text, main_infos[2].text, main_infos[3].text
        
        # Selecionando modalidade
        main_modalidade = driver.find_element_by_xpath('//div[@class="offer-card__info-text"]').text.split('\n')[-1]
        # Selecionando local
        main_local = driver.find_element_by_xpath('//div[@class="campus-map__address-box"]/p').text
        
        # Inputando as informações acima no dicionário
        info_dic['ies'].append(ies)
        info_dic['curso'].append(curso)
        info_dic['modalidade'].append(main_modalidade)
        info_dic['local'].append(main_local)
        info_dic['sem_desconto'].append(main_preco_sem_desconto)
        info_dic['com_desconto'].append(main_preco_com_desconto)
        info_dic['inicio'].append(main_inicio)
        info_dic['turno'].append(main_turno)
        info_dic['duracao'].append(main_duracao)
        info_dic['nivel'].append(main_nivel)

        
        # Caso haja o link para a página de avaliação do curso irá adiciona-lo na lista link_avaliacao
        try:
            link_avalicao = driver.find_element_by_xpath('//div[@class="ratings-box__rating"]/a[@href]').get_attribute('href')
            avaliacao.append(link_avaliacao)
        except:
            pass
        
    except:
        not_passed.append(url)
        
    driver.quit()
    
    return not_passed, avaliacao, info_dic

In [ ]:
def call_all(combined):
    
    '''Função para chamar todas as funções acima'''
    
    for comb in combined:
        inputing_values('unip', combined)
        
    for link in links_to_crawl:
        link_list = get_links_quero_bolsa(link)
    
    for url in link_list:
        get_data(url)